In [17]:
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow as tf

# Define your data folders
train_data_folder = 'new Multi Conditions database'

# Define parameters
batch_size = 32
img_size = (32, 32)
epochs = 10

# Create an ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% training, 20% validation
)

# Add Keras layers for additional data augmentation
data_augmentation = keras.Sequential([
    ##preprocessing.Rescaling(1./255),
    preprocessing.RandomContrast(factor=0.2),
    preprocessing.RandomZoom(height_factor=0.2, width_factor=0.2),
    ##preprocessing.Rescaling(255),  # Rescale back to the original range before GaussianNoise
    tf.keras.layers.GaussianNoise(stddev=0.01),
    ##preprocessing.Rescaling(1./255),  # Final rescaling before feeding to the model
    preprocessing.RandomRotation(factor=0.1)  # Slight rotation
])

# Load and augment the training data
train_generator = datagen.flow_from_directory(
    train_data_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load and augment the validation data
validation_generator = datagen.flow_from_directory(
    train_data_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Count the number of classes
num_classes = len(train_generator.class_indices)
print(num_classes)

# Build the CNN model
model = keras.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3), padding='same'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Build the model with data augmentation
augmented_model = keras.Sequential([
    data_augmentation,
    layers.Lambda(random_brightness),
    model
])

# Compile the augmented model
augmented_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = augmented_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Save the model
augmented_model.save('your_model_with_augmentation.h5')



Found 21975 images belonging to 5 classes.
Found 5492 images belonging to 5 classes.
5
Epoch 1/10
687/687 [==============================] - 169s 245ms/step - loss: 1.0317 - accuracy: 0.5531 - val_loss: 0.8098 - val_accuracy: 0.6830
Epoch 2/10
687/687 [==============================] - 167s 243ms/step - loss: 0.8583 - accuracy: 0.6441 - val_loss: 0.7602 - val_accuracy: 0.6890
Epoch 3/10
687/687 [==============================] - 168s 244ms/step - loss: 0.7897 - accuracy: 0.6740 - val_loss: 0.7216 - val_accuracy: 0.7047
Epoch 4/10
687/687 [==============================] - 166s 242ms/step - loss: 0.7551 - accuracy: 0.6922 - val_loss: 0.7386 - val_accuracy: 0.7079
Epoch 5/10
687/687 [==============================] - 174s 253ms/step - loss: 0.7371 - accuracy: 0.6946 - val_loss: 0.6946 - val_accuracy: 0.7101
Epoch 6/10
687/687 [==============================] - 5850s 9s/step - loss: 0.7125 - accuracy: 0.7050 - val_loss: 0.7166 - val_accuracy: 0.7129
Epoch 7/10
687/687 [===================

In [5]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Lost')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('multi x-ray loss graph.png')
plt.show()